In [27]:
import pandas as pd
import re

# --- CSV読み込み ---
df = pd.read_csv("../dataset/all_sales_data_2021_2025.csv", encoding="utf-8-sig")

# --- 季節ラベル抽出関数 ---
def label_season(text):
    text = str(text).lower()
    if any(k in text for k in ['春 夏 秋 冬', 'オールシーズン', '通年']):
        return '通年'
    elif any(k in text for k in ['秋冬', '秋 冬']):
        return '秋冬'
    elif any(k in text for k in ['春秋', '春 秋', '秋春', '秋 春']):
        return '春秋'
    elif any(k in text for k in ['春夏', '春 夏']):
        return '春夏'
    return ''

# --- カテゴリラベル抽出関数 ---
def label_category(text):
    text = str(text).lower()
    category_keywords = {
        'Outer': ['コート', 'ジャケット', 'ブルゾン', 'アウター', 'ダウン', 'パーカー'],
        'Tops': ['トップス', 'ブラウス', 'シャツ', 'ニット', 'カーディガン', 'カットソー', 'セーター', 'tシャツ'],
        'Bottoms': ['パンツ', 'スカート', 'ボトムス', 'ガウチョ', 'チュニック'],
        'Dress': ['ワンピース', 'ドレス'],
        'Suit': ['スーツ', 'セットアップ', 'ジャケット＆パンツ', '上下セット'],
        'Accessory': ['バッグ', '帽子', 'ハット', 'ベルト', 'マフラー', 'ストール', '手袋', 'アクセサリー', 'スカーフ']
    }
    for cat, keywords in category_keywords.items():
        if any(k in text for k in keywords):
            return cat
    return ''

# --- ブランド名候補抽出関数 ---
def extract_brand_candidate(text):
    if pd.isna(text):
        return ''
    
    text = str(text)

    # コンディションワード + 装飾記号の除去（先頭にあるものをまとめて）
    text = re.sub(r'^(【.*?】|[☆★◆■]*[A-Z]*[美新超良極HMSL]?品[同相様]?|未使用|新品同様|新品同様/|中古)+[/＊*♪☆★◆■\s-]*', '', text).strip()

    # トークンに分割（記号・スペース・日本語の区切り対応）
    tokens = re.split(r'[＊*/\s♪☆★◆■【】・]+', text)

    # 前から2〜3語を候補として返す（英語名＋カタカナなどに対応しやすく）
    tokens = [t for t in tokens if t.strip()]
    return " ".join(tokens[:3]) if tokens else ''

# --- ラベル付与 ---
df['季節ラベル'] = df['Title'].apply(label_season)
df['カテゴリラベル'] = df['Title'].apply(label_category)
df['ブランド名候補'] = df['Title'].apply(extract_brand_candidate)

# --- 確認 ---
# display(df[['Title', 'ブランド名候補', '季節ラベル', 'カテゴリラベル']])

# --- 保存（必要に応じて） ---
df.to_csv("../dataset/all_sales_data_2021_2025_labeled.csv", index=False, encoding="utf-8-sig")

df


,OrderTime,Title,TotalPrice,TotalMallCouponDiscount,ShipCharge,季節ラベル,カテゴリラベル,ブランド名候補
0,2021-06-16,美品/東京ソワール D'REREAD ブラックフォーマルワンピース 大きいサイズ 表記15号...,27720,0.0,0,春夏,Dress,東京ソワール D'REREAD ブラックフォーマルワンピース
1,2021-06-20,【中古】インディヴィ INDIVI 暖かウール混コート/ジャケット 表記38号(9号/M相当...,20000,0.0,0,秋冬,Outer,インディヴィ INDIVI 暖かウール混コート
2,2021-06-22,【中古】23区 美形パンツスーツ 表記40号(11号/L相当) グレー 薄手 毛ウール混 シ...,18000,0.0,0,春夏,Bottoms,23区 美形パンツスーツ 表記40号(11号
3,2021-06-24,【中古】クリスチャンオジャール CHRISTIAN AUJARD 茶ブラウン 7分袖 美柄カ...,7200,0.0,0,春夏,Tops,クリスチャンオジャール CHRISTIAN AUJARD
4,2021-07-02,【中古】斉藤都世子 toyoko saito ニットの女王 大人ニットワンピース 9号/38...,39500,500.0,0,秋冬,Tops,斉藤都世子 toyoko saito
...,...,...,...,...,...,...,...,...
2558,2025-04-12,美品【中古】バーバリー BURBERRYS スカート 台形スカート 表記 11号 L 40号...,16300,0.0,0,春夏,Bottoms,バーバリー BURBERRYS スカート
2559,2025-04-12,美品/レリアン Leilian フレアスカート 表記 11号 L 相当 黒 ブラック 毛 ウ...,15400,0.0,0,春秋,Bottoms,レリアン Leilian フレアスカート
2560,2025-04-14,美品/ユキトリイ YUKI TORII フォーマルジャケット 9号 M 相当 白 ホワイト ...,28600,0.0,0,春秋,Outer,ユキトリイ YUKI TORII
2561,2025-04-15,A美品/B3/ビースリー/白/ストレッチ素材/ロゴ釦/茶ステッチライン/お洒落カプリパンツ/...,3800,0.0,600,春夏,Bottoms,B3 ビースリー 白


In [26]:
from rapidfuzz import process, fuzz
import pandas as pd
import re, unicodedata

# -- 正規化ヘルパー --------------------------------
def normalize(txt: str) -> str:
    if pd.isna(txt): return ""
    txt = unicodedata.normalize("NFKC", txt)
    return re.sub(r"\s+", " ", txt).strip()

# -- ブランド辞書読み込み --------------------------
brands = [b.strip() for b in open("../dicts/brands.txt", encoding="utf-8") if b.strip()]

# -- 抽出関数 --------------------------------------
def fuzzy_extract_brand(title: str, score_cutoff: int = 80) -> str:
    t = normalize(title)
    # RapidFuzz で最も類似度の高い候補を1つ取る
    match = process.extractOne(
        t,                 # 検索対象文字列
        brands,            # 辞書リスト
        scorer=fuzz.partial_ratio,
        score_cutoff=score_cutoff,
    )
    # match = (best_brand, score, index) の形
    return match[0] if match else ""

# -- 動作確認 --------------------------------------
samples = [
    "美品◇Burberry トレンチコート",
    "新品同様 コムサデモード ニットワンピース",
    "ハイブランド Louis Vuitton バッグ",
    "おしゃれエルメス スカーフ",
    "お手頃GUデニムパンツ",
]
for s in samples:
    print(s, "→", fuzzy_extract_brand(s))


美品◇Burberry トレンチコート → チコ
新品同様 コムサデモード ニットワンピース → トワ
ハイブランド Louis Vuitton バッグ → ハケイ
おしゃれエルメス スカーフ → エルメス
お手頃GUデニムパンツ → 


In [23]:
samples = [
    "美品◇Burberry トレンチコート",
    "新品同様 コムサデモード ニットワンピース",
    "ハイブランド Louis Vuitton バッグ",
]
for s in samples:
    print(s, "→", ner_brand_with_ruler(s))
# 期待:
# 美品◇Burberry トレンチコート → Burberry
# 新品同様 コムサデモード ニットワンピース → コムサデモード
# ハイブランド Louis Vuitton バッグ → Louis Vuitton


美品◇Burberry トレンチコート → 
新品同様 コムサデモード ニットワンピース → 
ハイブランド Louis Vuitton バッグ → 
